# Overview

Similar to other examples that we had in the course - this notebook will demo how to scrape [Books to scrape](https://books.toscrape.com/index.html) - a fictional demo site that shows book prices and is designed for teaching web scraping! As it looks like a retailer webiste - we can scrape this site easily! 

In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd 